In [55]:
from openai import OpenAI
import numpy as np
import faiss
import json
from tqdm import tqdm

In [59]:
api_key = "API-key"

In [66]:
# Шаг 1: Функция для генерации эмбеддингов через OpenAI API
def generate_openai_embeddings(texts, model="text-embedding-ada-002", batch_size=10):
    embeddings = []
    client = OpenAI(
        api_key=api_key
    )
    for i in tqdm(range(0, len(texts), batch_size), desc="Генерация эмбеддингов"):
        batch_texts = texts[i:i + batch_size]
        response = client.embeddings.create(
            input=batch_texts,
            model=model
        )
        batch_embeddings = [embedding.embedding for embedding in response.data]
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [67]:
# Шаг 2: Загрузка данных
json_file_path = '../data/cleaned/combined_news.json'
faiss_index_path = '../data/cleaned/news_faiss_openai.index'

with open(json_file_path, 'r', encoding='utf-8') as file:
    news_data = json.load(file)

texts = [item['text'] for item in news_data]
metadata = [{'datetime': item['datetime'], 'link': item['link']} for item in news_data]

In [68]:
# Шаг 3: Генерация эмбеддингов с помощью OpenAI
embeddings = generate_openai_embeddings(texts, model="text-embedding-ada-002")

Генерация эмбеддингов: 100%|██████████| 1121/1121 [14:17<00:00,  1.31it/s]


In [69]:
# Шаг 4: Нормализация эмбеддингов (для косинусной схожести)
def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

normalized_embeddings = normalize_embeddings(embeddings)

In [70]:
# Шаг 5: Создание FAISS-индекса
dimension = normalized_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product для косинусной схожести
index.add(normalized_embeddings)

In [72]:
# Шаг 6: Сохранение индекса и метаданных
faiss.write_index(index, faiss_index_path)
with open('../data/cleaned/news_metadata.json', 'w', encoding='utf-8') as metadata_file:
    json.dump(metadata, metadata_file, ensure_ascii=False, indent=4)

print(f'Индекс сохранен в {faiss_index_path}')
print('Метаданные сохранены в data/cleaned/news_metadata.json')

Индекс сохранен в ../data/cleaned/news_faiss_openai.index
Метаданные сохранены в data/cleaned/news_metadata.json


In [78]:
def search_faiss_with_openai(query, model, index, metadata, top_k=5):
    # Генерация эмбеддинга для запроса через OpenAI
    client = OpenAI(
        api_key=api_key
    )
    response = client.embeddings.create(
            input=query,
            model=model
        )

    query_embedding = [embedding.embedding for embedding in response.data]
    
    # Нормализация эмбеддинга запроса
    query_embedding = normalize_embeddings(query_embedding)
    
    # Поиск в FAISS
    distances, indices = index.search(query_embedding, top_k)
    
    # Формирование результатов
    results = []
    for idx in indices[0]:
        if idx != -1:
            results.append({
                'text': texts[idx],  # Извлекаем текст
                'datetime': metadata[idx]['datetime'],  # Метаданные: дата
                'link': metadata[idx]['link']  # Метаданные: ссылка
            })
    return results

# Пример запроса
query = "новости из зоопарка"
results = search_faiss_with_openai(query, "text-embedding-ada-002", index, metadata)

# Вывод результатов
for res in results:
    print(f"Текст: {res['text']}")
    print(f"Дата: {res['datetime']}")
    print(f"Ссылка: {res['link']}")
    print("-" * 50)

Текст: Самым просматриваемым животным на онлайн-трансляциях Московского зоопарка стала панда Катюша.Об этом рассказал заместитель гендира Московского зоопарка по зоологическим вопросам Суслов.С 30 сентября на сайте мэра и правительства Москвы можно наблюдать за обитателями зоопарка, трансляция ведется в часы его работы.
Дата: None
Ссылка: https://t.me/rian_ru/265376
--------------------------------------------------
Текст: Ленинградский зоопарк публикует новые кадры из жизниродившегосяв мае 2023 года манула Шу, который готовится к зиме и продолжает активную зажировку
Дата: None
Ссылка: https://t.me/rian_ru/264987
--------------------------------------------------
Текст: Обвиняемый в краже кенгуру из ярославского зоопарка арестован, передает корреспондент РИА Новости из суда
Дата: 04-10-2024 09:54:00
Ссылка: https://t.me/rian_ru/263956
--------------------------------------------------
Текст: Полиция нашла львенка в пригороде Ростова-на-Дону, его привезли в местный зоопарк.Малышу пример